# Verification de l'implementation de la CL Echange_Externe_Radiatif

In [ ]:
from trustutils import run
import numpy as np

run.introduction("Elie Saikali & Yannick Gorsse")

run.TRUST_parameters()
run.reset()

Text = 500
eps = 0.7

parameters = {
    "Text": Text,
    "eps": eps,
    "dt" : 1,
}
dis = {"VDF" : "", "VEF" : "trianguler_h dom"}
fluxes = np.linspace(0, 1e6, 10)

nb_proc = 1
for d, mesh in dis.items():
    for flux in fluxes:
        parameters["flux"] = flux
        parameters["mesh"] = mesh
        parameters["dis"] = d
        tc = run.addCaseFromTemplate(f"jdd.data", f"{d}/flux_{flux:.1e}", {**parameters}, nb_proc)
        if nb_proc > 1: tc.partition()
run.printCases()
run.runCases()
run.tablePerf()

In [ ]:
from trustutils import plot

a = plot.Graph()
for d in dis:
    for flux in fluxes:
        a.addResidu(f"{run.BUILD_DIRECTORY}/{d}/flux_{flux:.1e}/jdd.dt_ev", label=f"{d} - flux_{flux:.1e}")

a.scale(yscale="log")

In [ ]:
a = plot.Graph()
for d in dis:
    temperatures = []
    exact = []
    for flux in fluxes:
        T = plot.loadText(f"{run.BUILD_DIRECTORY}/{d}/flux_{flux:.1e}/jdd_T.son")[-1][-1]
        temperatures.append(T)
        exact.append(pow(flux / eps / 5.67e-8 + Text**4, 0.25))

    a.add(fluxes, temperatures, label=d)
a.add(fluxes, exact, label="Analytique", marker="o")

a = plot.Graph()
for d in dis:
    a.addSegment(f"{run.BUILD_DIRECTORY}/{d}/flux_{fluxes[-1]:.1e}/jdd_T_PROF.son", label=d, marker="o" if d =="VDF" else "-")